#### Load Package

In [1]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd
import pickle
from pathlib import Path
from itertools import product

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn

import mlflow

#### Setup

In [2]:
DATA_DIR = "/Users/kuldeepsharma/github/mlops/iris-mlops/data/"
RAW_DATA_PATH = DATA_DIR + "raw_data.csv"
PROCESSED_DATA_PATH = DATA_DIR + "processed_data.csv"
MODEL_PATH = "/Users/kuldeepsharma/github/mlops/iris-mlops/models/"

#### Extract Data

In [3]:
data = pd.read_csv(RAW_DATA_PATH)
data.head(3)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,52,6.4,3.2,4.5,1.5,Iris-versicolor
1,116,6.4,3.2,5.3,2.3,Iris-virginica
2,23,4.6,3.6,1.0,0.2,Iris-setosa


#### Transform Data

In [4]:
### Remove 'Id' Column
data = data.drop(columns=['Id'], axis=1)

### Rename Columns
data.columns = ["sepal_lenght", "sepal_width", "petal_lenght",
                    "petal_width", "target"]

### Encode Labels
data["target"] = data["target"].map(
        {"Iris-setosa": 0, "Iris-versicolor": 1, "Iris-virginica": 2}
    )

### Drop Duplicates
data = data.drop_duplicates()

data.head(3)

,sepal_lenght,sepal_width,petal_lenght,petal_width,target
0,6.4,3.2,4.5,1.5,1
1,6.4,3.2,5.3,2.3,2
2,4.6,3.6,1.0,0.2,0


#### Load Data

In [5]:
data.to_csv(PROCESSED_DATA_PATH, index=False)

#### Prepara Data

In [6]:
### Split data in train, val and test
X = data.drop(columns=['target']).values
y = data['target'].values

X_train, X_temp, y_train, y_temp =\
    train_test_split(X, y, test_size=0.3, stratify=y, random_state=3)
X_val, X_test, y_val, y_test =\
    train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=3)

### Convert to tensors
X_train = torch.FloatTensor(X_train)
X_val = torch.FloatTensor(X_val)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_val = torch.LongTensor(y_val)
y_test = torch.LongTensor(y_test)

#### Model & Parameters

In [7]:
class SimpleNeuralNetwork(nn.Module):
    def __init__(self, input_dim, output_dim,
            layer1_dim=128, layer2_dim=64, act="relu"):
        
        super(SimpleNeuralNetwork,self).__init__()
        self.input_layer    = nn.Linear(input_dim, layer1_dim)
        self.hidden_layer1  = nn.Linear(layer1_dim, layer2_dim)
        self.output_layer   = nn.Linear(layer2_dim, output_dim)
        if act == "relu":
            self.act = nn.ReLU()
        if act == "tanh":
            self.act = nn.Tanh()
    
    def forward(self, x):
        x =  self.act(self.input_layer(x))
        x =  self.act(self.hidden_layer1(x))
        x =  self.output_layer(x)
        return x
    

### Parameters
num_epochs = 100
input_dim  = 4 
output_dim = 3
model = SimpleNeuralNetwork(input_dim, output_dim, 
            layer1_dim=128, layer2_dim=64, act="relu")

# Creating our optimizer and loss function
learning_rate = 0.01
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#### Train & Validate

In [8]:
def train_n_validate_model(model, optimizer, criterion,
    X_train, y_train, X_val, y_val, num_epochs):
    
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()

        output_train = model(X_train)
        loss_train = criterion(output_train, y_train)
        loss_train.backward()
        optimizer.step()

        loss_train = loss_train.item()
        _, predicted_train = torch.max(output_train, 1)
        correct_predictions_train = (predicted_train == y_train).sum().item()
        total_train_samples = y_train.size(0) * 1.0
        
        model.eval()
        output_val = model(X_val)
        loss_val = criterion(output_val, y_val)
        _, predicted_val = torch.max(output_val, 1)
        correct_predictions_val = (predicted_val == y_val).sum().item()
        total_val_samples = y_val.size(0) * 1.0

        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}")
            print(f"Train Loss: {loss_train:.4f}")
            print(f"Train Accuracy: {correct_predictions_train/total_train_samples:.4f}")
            print(f"Val Loss: {loss_val:.4f}")
            print(f"Val Accuracy: {correct_predictions_val/total_val_samples:.4f}")

train_n_validate_model(model, optimizer, criterion, X_train, y_train, X_val, y_val, num_epochs)

Epoch 10/100
Train Loss: 0.3722
Train Accuracy: 0.8675
Val Loss: 0.3059
Val Accuracy: 0.9444
Epoch 20/100
Train Loss: 0.1353
Train Accuracy: 0.9639
Val Loss: 0.0781
Val Accuracy: 1.0000
Epoch 30/100
Train Loss: 0.0616
Train Accuracy: 0.9759
Val Loss: 0.0262
Val Accuracy: 1.0000
Epoch 40/100
Train Loss: 0.0414
Train Accuracy: 0.9880
Val Loss: 0.0224
Val Accuracy: 1.0000
Epoch 50/100
Train Loss: 0.0336
Train Accuracy: 0.9880
Val Loss: 0.0143
Val Accuracy: 1.0000
Epoch 60/100
Train Loss: 0.0287
Train Accuracy: 0.9759
Val Loss: 0.0108
Val Accuracy: 1.0000
Epoch 70/100
Train Loss: 0.0256
Train Accuracy: 1.0000
Val Loss: 0.0079
Val Accuracy: 1.0000
Epoch 80/100
Train Loss: 0.0234
Train Accuracy: 1.0000
Val Loss: 0.0072
Val Accuracy: 1.0000
Epoch 90/100
Train Loss: 0.0214
Train Accuracy: 1.0000
Val Loss: 0.0051
Val Accuracy: 1.0000
Epoch 100/100
Train Loss: 0.0244
Train Accuracy: 1.0000
Val Loss: 0.0010
Val Accuracy: 1.0000


#### Test Model

In [9]:
def test_model(model, X_test, y_test):
    
    model.eval()

    output_test = model(X_test)
    _, predicted_test = torch.max(output_test, 1)
    correct_predictions_test = (predicted_test == y_test).sum().item()
    total_test_samples = y_test.size(0) * 1.0
    accuracy = round(correct_predictions_test / total_test_samples, 4)
    print(f"Test Accuracy: {accuracy * 100.0} %")
    return accuracy * 100.0

test_model(model, X_test, y_test)

Test Accuracy: 94.44 %


94.44

#### Experiments Tracking

In [10]:
# Config MLflow
MODEL_REGISTRY = Path("/tmp/mlflow")
Path(MODEL_REGISTRY).mkdir(parents=True, exist_ok=True)
MLFLOW_TRACKING_URI = "file://" + str(MODEL_REGISTRY.absolute())
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [11]:
# Create or get an experiment
experiment_name = "iris-notebook-experiments"
mlflow.set_experiment(experiment_name)

<Experiment: artifact_location='file:///tmp/mlflow/589294911647954324', creation_time=1691308017249, experiment_id='589294911647954324', last_update_time=1691308017249, lifecycle_stage='active', name='iris-notebook-experiments', tags={}>

In [12]:
def train_n_validate_model_mlflow(model, optimizer, criterion,
    X_train, y_train, X_val, y_val, num_epochs, train_losses, val_losses):

    
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()

        output_train = model(X_train)
        loss_train = criterion(output_train, y_train)
        loss_train.backward()
        optimizer.step()

        loss_train = loss_train.item()
        train_losses[epoch] = loss_train
        
        model.eval()
        output_val = model(X_val)
        loss_val = criterion(output_val, y_val)

        loss_val = loss_val.item()
        val_losses[epoch] = loss_val

In [13]:
### Parameters
input_dim  = 4 
output_dim = 3

num_epochs = [10, 50, 100]
learning_rates = [0.1, 0.01]
layer1_dims = [64, 128, 256]
layer2_dims = [32, 64, 128]
activation_functions = ["relu"]

combinations = product(num_epochs, learning_rates,
                    layer1_dims, layer2_dims, activation_functions)

for i, combination in enumerate(combinations):
    num_epochs, lr, l1_dim, l2_dim, act = combination

    model = SimpleNeuralNetwork(input_dim=input_dim, output_dim=output_dim,
                                layer1_dim=l1_dim, layer2_dim=l2_dim, act=act)


    learning_rate = lr
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    train_losses = np.zeros(num_epochs)
    val_losses = np.zeros(num_epochs)

    train_n_validate_model_mlflow(model, optimizer, criterion,
        X_train, y_train, X_val, y_val, num_epochs, train_losses, val_losses)

    with mlflow.start_run(run_name=f"run-1.{i}", description=f"version-1") as run:
        params = {
            "input_dim": input_dim,
            "output_dim": output_dim,
            "num_epochs": num_epochs,
            "learning_rate": learning_rate,
            "layer1_dim": l1_dim,
            "layer2_dim": l2_dim,
            "activation_function": act,
        }
        mlflow.log_params(params=params)

        mlflow.pytorch.log_model(model, f"iris-pytorch-model")

        for i in range(len(list(train_losses))):
            mlflow.log_metrics({"train_loss": list(train_losses)[i]}, step=i)
            mlflow.log_metrics({"val_loss": list(val_losses)[i]}, step=i)

        accuracy = test_model(model, X_test, y_test)
        mlflow.log_metrics({"accuracy": accuracy})

Test Accuracy: 88.89 %
Test Accuracy: 94.44 %
Test Accuracy: 88.89 %
Test Accuracy: 88.89 %
Test Accuracy: 88.89 %
Test Accuracy: 83.33 %
Test Accuracy: 66.67 %
Test Accuracy: 66.67 %
Test Accuracy: 33.33 %
Test Accuracy: 88.89 %
Test Accuracy: 94.44 %
Test Accuracy: 88.89 %
Test Accuracy: 94.44 %
Test Accuracy: 88.89 %
Test Accuracy: 88.89 %
Test Accuracy: 94.44 %
Test Accuracy: 66.67 %
Test Accuracy: 83.33 %
Test Accuracy: 100.0 %
Test Accuracy: 94.44 %
Test Accuracy: 66.67 %
Test Accuracy: 94.44 %
Test Accuracy: 94.44 %
Test Accuracy: 94.44 %
Test Accuracy: 94.44 %
Test Accuracy: 94.44 %
Test Accuracy: 33.33 %
Test Accuracy: 100.0 %
Test Accuracy: 100.0 %
Test Accuracy: 94.44 %
Test Accuracy: 94.44 %
Test Accuracy: 100.0 %
Test Accuracy: 100.0 %
Test Accuracy: 100.0 %
Test Accuracy: 94.44 %
Test Accuracy: 94.44 %
Test Accuracy: 94.44 %
Test Accuracy: 94.44 %
Test Accuracy: 94.44 %
Test Accuracy: 94.44 %
Test Accuracy: 94.44 %
Test Accuracy: 94.44 %
Test Accuracy: 94.44 %
Test Accura

In [14]:
# Search for runs in the experiment, ordered by validation loss
sorted_runs = mlflow.search_runs(experiment_names=[experiment_name],
                                    order_by=["metrics.val_loss ASC"])
if not sorted_runs.empty:
    best_run_id = sorted_runs.iloc[0].run_id
    # Load the best model using the artifact URI
    best_model = mlflow.pytorch.load_model(f"runs:/{best_run_id}/iris-pytorch-model")

    # Now you can use the best_model for inference or evaluation
    accuracy = test_model(best_model, X_test, y_test)

Test Accuracy: 94.44 %


In [85]:
import sys
sys.path.append("./../src")

from utils import mlflow_config

mlflow = mlflow_config()

try:
    production_model_run_id = mlflow.models.get_model_info(
                f"models:/iris-production-model/Production"
            ).run_id
except:
    print("Please go to mlflow ui and cretae a production model registry manually :(")

Please go to mlflow ui and cretae a production model registry manually :(
